Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [20]:
# Logistic regression with regularization

batch_size = 128
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta  regularizer)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

SyntaxError: invalid syntax (<ipython-input-20-0b377efdc6b1>, line 27)

In [21]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 30.345863
Minibatch accuracy: 8.6%
Validation accuracy: 9.9%
Minibatch loss at step 500: 2.712748
Minibatch accuracy: 78.1%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 1.010251
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 0.812974
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 0.741093
Minibatch accuracy: 82.0%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 0.837717
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.815192
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 3500: 0.794336
Minibatch accuracy: 80.5%
Validation accuracy: 81.1%
Minibatch loss at step 4000: 0.694464
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 4500: 0.615053
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 5000: 0.722000
Minibatch accuracy: 78.1%
Validation accuracy

In [25]:
# Neural net with one hidden layer and regularization
batch_size = 128
beta = 0.03

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  b1 = tf.Variable(tf.zeros([1024]))
  w2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  logits = tf.matmul(hidden_layer, w2) + b2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
  loss = tf.reduce_mean(loss1 + beta * regularizer)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [26]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3489.204102
Minibatch accuracy: 14.1%
Validation accuracy: 30.3%
Minibatch loss at step 500: 21.277126
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.856317
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.787606
Minibatch accuracy: 82.8%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.762528
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.797503
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.867183
Minibatch accuracy: 79.7%
Validation accuracy: 83.1%
Minibatch loss at step 3500: 0.751368
Minibatch accuracy: 82.8%
Validation accuracy: 83.6%
Minibatch loss at step 4000: 0.701215
Minibatch accuracy: 85.2%
Validation accuracy: 83.4%
Minibatch loss at step 4500: 0.665239
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 5000: 0.653319
Minibatch accuracy: 82.8%
Validation acc

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
num_steps = 10000
small_train_dataset = train_dataset[:512, :]
small_train_labels = train_labels[:512]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1295.932983
Minibatch accuracy: 8.6%
Validation accuracy: 29.7%
Minibatch loss at step 500: 210.132828
Minibatch accuracy: 100.0%
Validation accuracy: 75.3%
Minibatch loss at step 1000: 46.836384
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 1500: 10.472178
Minibatch accuracy: 100.0%
Validation accuracy: 76.1%
Minibatch loss at step 2000: 2.416728
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2500: 0.635693
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 3000: 0.233769
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 3500: 0.139497
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 4000: 0.119001
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 4500: 0.111243
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 5000: 0.106850
Minibatch accuracy: 100.0%
Va

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [44]:
# Neural net with one hidden layer and regularization and dropout
batch_size = 128
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  b1 = tf.Variable(tf.zeros([1024]))
  w2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  hidden_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
  logits = tf.matmul(hidden_layer_dropout, w2) + b2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
  loss = tf.reduce_mean(loss1 + beta * regularizer)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [45]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1523.603394
Minibatch accuracy: 9.4%
Validation accuracy: 36.0%
Minibatch loss at step 500: 214.100723
Minibatch accuracy: 71.9%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 46.377853
Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 10.958666
Minibatch accuracy: 75.8%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 3.044969
Minibatch accuracy: 82.0%
Validation accuracy: 85.1%


KeyboardInterrupt: 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [43]:
# Neural net with two hidden layer and regularization and dropout
batch_size = 128
beta = 0.0

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.'
  w1 = tf.get_variable('w1', shape=[image_size * image_size, 1024], initializer=tf.contrib.layers.xavier_initializer())
  b1 = tf.Variable(tf.zeros([1024]))
  w2 = tf.get_variable('w2', shape=[1024, 512], initializer=tf.contrib.layers.xavier_initializer())
  b2 = tf.Variable(tf.zeros([512]))
  w3 = tf.get_variable('w3', shape=[512, 256], initializer=tf.contrib.layers.xavier_initializer())
  b3 = tf.Variable(tf.zeros([256]))
  w4 = tf.get_variable('w4', shape=[256, 128], initializer=tf.contrib.layers.xavier_initializer())
  b4 = tf.Variable(tf.zeros([128]))
  w5 = tf.get_variable('w5', shape=[128, 64], initializer=tf.contrib.layers.xavier_initializer())
  b5 = tf.Variable(tf.zeros([64]))
  w6 = tf.get_variable('w6', shape=[64, 32], initializer=tf.contrib.layers.xavier_initializer())
  b6 = tf.Variable(tf.zeros([32]))
  w7 = tf.get_variable('w7', shape=[32, num_labels], initializer=tf.contrib.layers.xavier_initializer())
  b7 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  fc1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  fc1_drop = tf.nn.dropout(fc1, keep_prob)
    
  fc2 = tf.nn.relu(tf.matmul(fc1_drop, w2) + b2)
  fc2_drop = tf.nn.dropout(fc2, keep_prob)
    
  fc3 = tf.nn.relu(tf.matmul(fc2_drop, w3) + b3)
  fc3_drop = tf.nn.dropout(fc3, keep_prob)
    
  fc4 = tf.nn.relu(tf.matmul(fc3_drop, w4) + b4)
  fc4_drop = tf.nn.dropout(fc4, keep_prob)
    
  fc5 = tf.nn.relu(tf.matmul(fc4_drop, w5) + b5)
  fc5_drop = tf.nn.dropout(fc5, keep_prob)
    
  fc6 = tf.nn.relu(tf.matmul(fc5_drop, w6) + b6)
  fc6_drop = tf.nn.dropout(fc6, keep_prob)
  
  logits = tf.matmul(fc6_drop, w7) + b7
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + tf.nn.l2_loss(w4)+ tf.nn.l2_loss(w5)  + tf.nn.l2_loss(w6)  + tf.nn.l2_loss(w7)
  loss = tf.reduce_mean(loss + beta * regularizer)
  
  # Optimizer.
  global_step = tf.Variable(0)
  starter_learning_rate = 0.001
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
  

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  fc1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)    
  fc2_valid = tf.nn.relu(tf.matmul(fc1_valid, w2) + b2)    
  fc3_valid = tf.nn.relu(tf.matmul(fc2_valid, w3) + b3)    
  fc4_valid = tf.nn.relu(tf.matmul(fc3_valid, w4) + b4)    
  fc5_valid = tf.nn.relu(tf.matmul(fc4_valid, w5) + b5)    
  fc6_valid = tf.nn.relu(tf.matmul(fc5_valid, w6) + b6)  
  logits_valid = tf.matmul(fc6_valid, w7) + b7
  valid_prediction = tf.nn.softmax(logits_valid)
    
  fc1_test = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)    
  fc2_test = tf.nn.relu(tf.matmul(fc1_test, w2) + b2)    
  fc3_test = tf.nn.relu(tf.matmul(fc2_test, w3) + b3)    
  fc4_test = tf.nn.relu(tf.matmul(fc3_test, w4) + b4)    
  fc5_test = tf.nn.relu(tf.matmul(fc4_test, w5) + b5)    
  fc6_test = tf.nn.relu(tf.matmul(fc5_test, w6) + b6)  
  logits_test = tf.matmul(fc6_test, w7) + b7
  test_prediction = tf.nn.softmax(logits_test)

In [ ]:
num_steps = 100000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.640917
Minibatch accuracy: 10.9%
Validation accuracy: 12.7%
Minibatch loss at step 500: 0.700667
Minibatch accuracy: 83.6%
Validation accuracy: 85.3%
Minibatch loss at step 1000: 0.593161
Minibatch accuracy: 88.3%
Validation accuracy: 86.6%
Minibatch loss at step 1500: 0.734397
Minibatch accuracy: 84.4%
Validation accuracy: 86.9%
Minibatch loss at step 2000: 0.645246
Minibatch accuracy: 85.9%
Validation accuracy: 87.9%
Minibatch loss at step 2500: 0.524202
Minibatch accuracy: 86.7%
Validation accuracy: 88.0%
Minibatch loss at step 3000: 0.652390
Minibatch accuracy: 83.6%
Validation accuracy: 88.2%
Minibatch loss at step 3500: 0.514780
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 4000: 0.516411
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
